In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# Settings the warnings to be ignored
warnings.filterwarnings('ignore')




#package สำหรับจัดการ html เรียกว่า BeautifulSoup

def get_soup(url):
    with requests.get(url, verify=False) as r:
        soup = BeautifulSoup(r.text, features='html.parser')
    return soup

class ku_scholar:
  def __init__ (self,url):
    self.head_link = 'https://iad.intaff.ku.ac.th/portal/'
    self.main_link = url
    self.soup = get_soup(self.main_link)
  
  def all_link(self): # get list link of university
    self.link = []
    self.name = []
    for url in self.soup.find_all('tr', class_='row-content'):
      try:
        self.tail_link = (url.find('a').get('href'))
        self.full_link = self.head_link + self.tail_link
        self.link.append(self.full_link)
      except:
        pass
    return self.link
  
  def search_name(self, name):
    self.name_input = name
    self.name_list = self.all_name()
    self.name_list = (list(set(self.name_list))) # convert to set avoid duplicated data
    self.name_list = sorted(self.name_list) # sort alphabet

    self.found_name = []
    self.found = 0

    for name in self.name_list:
      if self.name_input.lower() in name.lower():
          self.found_name.append(name)
          self.found += 1

    if self.found < 1:
        print(f'Error : "{self.name_input}" not found on database.')

    return self.found_name

  def search_detail(self, name): # search_datail (/all for get_all university data)
    self.name_input = name

    if self.name_input == '/all':
      print(f'getting all data.. ({len(self.all_name())} university)')

      self.df_main = self.get_detail(self.all_name()[0])
      
      for i in range(1, len(self.all_name())):
        try:
          self.df_main = self.df_main.append(self.get_detail(self.all_name()[i]))
          print(i ,end=" ")
          if i % 20 == 0:
            print("\n")
        except:
          pass
      
    else:
      self.university_list = self.search_name(name) # get university_name

      if len(self.university_list) > 0:
        if len(self.university_list) == 1:
          return self.get_detail(self.university_list[0])
        else:
          self.df_main = self.get_detail(self.university_list[0])
          for i in range(1, len(self.university_list)):
            try:
              self.df_main = self.df_main.append(self.get_detail(self.university_list[i]))
            except:
              pass
      else:
        print(f'Error ! :{self.name_input} is not found on database.')

    return self.df_main


  def all_name(self): # get list name of university
    self.link = []
    self.name = []
    for url in self.soup.find_all('tr', class_='row-content'):
        #name.append(url.text.strip())
        try:
          self.name.append(str(url.find('a').text.strip()))
        except:
          pass

        # debug duplicated name by add number out of name

    self.count_found = 0

    self.name_duplicated = []
    self.count_duplicated = []

    # Get_duplicated data in list

    for name in self.name:
      self.count_found = self.name.count(name)
      if self.count_found > 1:
        self.name_duplicated.append(name)
        self.count_duplicated.append(self.count_found)

    # Avoid duplicated value
    self.name_duplicated = list(set(self.name_duplicated))
    self.count_duplicated = list(set(self.count_duplicated))

    # rename duplicated item algorihm by using number append
    self.index = 0
    for count_val in (self.count_duplicated):
      for i in range(count_val):
        #self.df.loc[(self.name.index(self.name_duplicated[self.index]),'name')] = str(self.name_duplicated[self.index]) + "_" + str(i+1) # change name on pandas
        self.name[self.name.index(self.name_duplicated[self.index])] = str(self.name_duplicated[self.index]) + "_" + str(i+1) # change name on list_name
      self.index += 1


    return self.name


  def list_all(self): # return as pandas dataframe (name, link)
    self.link = []
    self.name = []
    for url in self.soup.find_all('tr', class_='row-content'):
        #name.append(url.text.strip())
        try:
          self.tail_link = (url.find('a').get('href'))
          self.full_link = self.head_link + self.tail_link
          self.link.append(self.full_link)
          self.name.append(str(url.find('a').text.strip()))
        except:
          pass

    self.df = pd.DataFrame({'name': self.name,'url': self.link})

    # debug duplicated name by add number out of name

    self.count_found = 0

    self.name_duplicated = []
    self.count_duplicated = []

    # Get_duplicated data in list

    for name in self.name:
      self.count_found = self.name.count(name)
      if self.count_found > 1:
        self.name_duplicated.append(name)
        self.count_duplicated.append(self.count_found)

    # Avoid duplicated value
    self.name_duplicated = list(set(self.name_duplicated))
    self.count_duplicated = list(set(self.count_duplicated))

    # rename duplicated item algorihm by using number append
    self.index = 0
    for count_val in (self.count_duplicated):
      for i in range(count_val):
        self.df.loc[(self.name.index(self.name_duplicated[self.index]),'name')] = str(self.name_duplicated[self.index]) + "_" + str(i+1) # change name on pandas
        self.name[self.name.index(self.name_duplicated[self.index])] = str(self.name_duplicated[self.index]) + "_" + str(i+1) # change name on list_name
        #print(name_duplicated[index], i+1)
      self.index += 1

    return self.df

  def get_url_link(self, university_name): # get url university link
    self.url = ""
    self.university_name = university_name
    self.data_list = self.list_all() # get name, url data
    try:
      self.url = (self.data_list.loc[self.data_list['name'] == self.university_name, 'url']).str.cat() # convert to plain str url
      return self.url
    except:
      print(f"Not found '{self.university_name}' in list")
  
  def get_csv_link(self, university_name): # get csv_url university link
    self.university_name = university_name
    self.main_url = self.get_url_link(self.university_name) # get university url
    self.csv_link = ""

    self.soup_csv = get_soup(self.main_url) # get soup from link

    for url in self.soup_csv.find('div', class_='dropdown fixed-position'):
      if 'csv' in str(url):
        self.csv_link = (url.get('href'))
        break

    return (self.head_link + self.csv_link).replace(" ", "%20") # debug space error
  
  def get_detail(self, university_name):
  
    self.university_name = university_name
    self.url_link = self.get_url_link(self.university_name) # get url link
    self.csv_link = self.get_csv_link(self.university_name) # get csv

    #pandas zone (csv)

    self.df = pd.read_csv(self.csv_link)

    self.dic = self.df.to_dict('series') # convert pandas to dict


    # debug error on csv

    for i in range(len(self.dic['No.'])):

      self.dic['Duration'][i] = str(self.dic['Duration'][i]).replace(' ','') # delete space on Duration Column
      if self.dic['MOU Level'][i] == "University": # del subordinating bug on University MOU level
        self.dic['Subordinating Unit of KU'][i] = 'NaN'
        self.dic['Subordinating Unit of Partner'][i] = 'NaN'
      if ("<br>" in str(self.dic['Subordinating Unit of KU'][i])):
        self.dic['Subordinating Unit of KU'][i] = str(self.dic['Subordinating Unit of KU'][i]).replace('<br>', '')


    # get auto - renew data

    self.soup_mou = get_soup(self.url_link)

    self.html_d = str(self.soup_mou.find('tr', class_='row-name')).replace(" ",'_').replace("-","") # convert to str and replace space to underspace
    self.soup_new = BeautifulSoup(self.html_d, 'html.parser') # convert to bs4 type
    #soup_new.text.split() # split_text using space

    self.html_m = str(self.soup_mou.find_all('tr', class_='row-content')).replace(" ",'_').replace('\t','').replace('\n','')
    #html_m


    self.html_m.split('</td>')

    self.pointer = False # Found glyphicon_glyphicon-ok True / False
    # ในกรณีที่ pointer = True จะมีคู่กันใน html (True กับ False เป็นเงื่อนไขคู่กัน แต่สนใจแค่ True) เลยตัดค่า False ออก 1 ตัว

    self.auto_renew = []

    for i in self.html_m.split('</td>'):
      if self.pointer == False:
        if 'glyphicon_glyphicon-ok' in i:
          if '<!--' not in i:
            self.auto_renew.append('Yes')
            self.pointer = True
            #break
          else:
            self.auto_renew.append('No')
            self.pointer = False
      else: # pointer == True | when pointer True skip 1 step
        self.pointer = False
        pass

    self.df_auto_new = pd.DataFrame({'Auto - Renewal' : self.auto_renew})


    self.result = pd.concat([pd.DataFrame.from_dict(self.dic), self.df_auto_new],axis=1)
    return self.result
      
  def count(self):
    return len(self.soup.find_all('tr', class_='row-content')) - 1 # remove labels column



In [2]:
ku = ku_scholar('https://iad.intaff.ku.ac.th/portal/mou_f021_2.php?ID1=&ID2=10')

In [3]:
ku.search_name('saitama')

['Saitama University']

In [4]:
ku.search_detail('saitama')

,No.,Name of Partners,MOU Name,MOU Level,Subordinating Unit of KU,Subordinating Unit of Partner,Effective Date,Expired Date,Duration,Auto - Renewal
0,1,Saitama University,Agreement on Academic Exchange,University,NaN,NaN,24/12/2564,23/12/2569,5Y,No
1,2,Saitama University,Memorandum on Student Exchange,University,NaN,NaN,24/12/2564,23/12/2569,5Y,No


In [ ]:
Data = ku_scholar()

In [ ]:
Data.all_name()

['Akita International University',
 'Akita Prefectural University',
 'Aomori Chuo Gakuin University',
 'Aoyama Gakuin University',
 'Chiba University',
 'Chuo University',
 'Forestry and Forest Products Research Institute',
 'Forica Foods Co., Ltd.',
 'Fukui Prefectural University',
 'Fukuoka University',
 'Gifu University',
 'Hiroshima Institute of Technology',
 'Hiroshima University',
 'Hokkaido University',
 'Ibaraki University',
 'Ishihara Sangyo Kaisha, Ltd.',
 'Iwate Prefectural University',
 'Iwate University',
 'Japan International Cooperation Agency (JICA)',
 'Japan International Research Center for Agricultural Science (JIRCAS)',
 'Japan Science and Technology Corporation (JST)',
 'Japanese Institute of Irrigation and Drainage',
 'Juntendo University',
 'Kagawa University',
 'Kagoshima National College of Technology',
 'Kagoshima University',
 'Kanazawa University',
 'Kansai University',
 'Kitasato University',
 'Kobe University',
 'Kobe Women s University',
 'Kochi Universit

In [ ]:
Data.search_name('tokyo')

['Tokyo Institute of Technology',
 'Tokyo University of Agriculture',
 'Tokyo University of Agriculture and Technology',
 'Tokyo University of Marine Science and Technology',
 'University of Tokyo']

In [ ]:
Data.search_detail('tokyo')

,No.,Name of Partners,MOU Name,MOU Level,Subordinating Unit of KU,Subordinating Unit of Partner,Effective Date,Expired Date,Duration,Auto - Renewal
0,1,Tokyo Institute of Technology,Cooperation Agreement,NaN,NaN,NaN,01/12/2539,01/12/2544,5Y,No
1,2,Tokyo Institute of Technology,Implementing Agreement for Promotion of Studen...,NaN,NaN,NaN,16/02/2544,unspecified,unspecified,No
2,3,Tokyo Institute of Technology,Extension Agreement : Cooperation Agreement,NaN,NaN,NaN,14/01/2545,25/12/2549,4Y,No
3,4,Tokyo Institute of Technology,Agreement for Cooperation,NaN,NaN,NaN,11/02/2551,25/12/2554,3Y,No
4,5,Tokyo Institute of Technology,TAIST Tokyo Tech Implementation Agreement,NaN,NaN,NaN,25/12/2554,24/12/2559,5Y,No
5,6,Tokyo Institute of Technology,Implement Agreement,NaN,NaN,NaN,12/03/2555,11/03/2560,5Y,No
6,7,Tokyo Institute of Technology,Extension Agreement: Agreement for Cooperation,NaN,NaN,NaN,26/12/2555,25/12/2559,4Y,No
7,8,Tokyo Institute of Technology,Extension Agreement: Cooperation Agreement,University,NaN,NaN,26/12/2564,24/06/2565,5Y,No
8,9,Tokyo Institute of Technology,Implementing Agreement for Promotion of Studen...,University,NaN,NaN,25/12/2564,24/12/2569,5Y,No
0,1,Tokyo University of Agriculture,Academic Exchange Agreement,NaN,NaN,NaN,09/06/2531,unspecified,unspecified,No


In [ ]:
Data.search_detail('/all')

getting all data.. (119 university)
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 

41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 

61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 

81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 

,No.,Name of Partners,MOU Name,MOU Level,Subordinating Unit of KU,Subordinating Unit of Partner,Effective Date,Expired Date,Duration,Auto - Renewal
0,1,Akita International University,Cooperation Agreement,University,NaN,NaN,18/10/2555,unspecified,unspecified,Yes
0,1,Akita Prefectural University,Memorandum of Understanding for Academic Coope...,Faculty,1. Faculty of Agriculture,1.Faculty of Bioresource Sciences,16/03/2555,15/03/2557,2Y,Yes
1,2,Akita Prefectural University,Memorandum of Understanding on Student /Schola...,Faculty,1. Faculty of Agriculture,1.Faculty of Bioresource Sciences,16/03/2557,15/03/2562,5Y,No
2,3,Akita Prefectural University,Academic Exchange Agreement,Faculty,1. Faculty of Agriculture,1.Faculty of Bioresource sciences,16/03/2557,15/03/2562,5Y,No
3,4,Akita Prefectural University,Academic Exchange Agreement,University,NaN,NaN,22/05/2562,21/05/2567,5Y,No
...,...,...,...,...,...,...,...,...,...,...
19,20,Yamaguchi University,Agreement for Establishment of Yamaguchi Unive...,University,NaN,NaN,01/04/2564,31/03/2565,1Y,No
0,1,Yokohama City University,Memorandum of Understanding,University,NaN,NaN,24/08/2559,23/08/2564,5Y,No
0,1,HR Japan,Memorandum of Understanding,Faculty,1. Faculty of Humanities,NaN,03/10/2561,02/10/2566,5Y,No
0,1,International Society of Environmental and Rur...,Memorandum of Agreement,University,NaN,NaN,03/09/2561,25/02/2562,nan,No
